# Variational Autoencoder using pytorch
Following the example from: <br>
https://medium.com/dataseries/variational-autoencoder-with-pytorch-2d359cbf027b

In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import random 
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# please specify data path
# datapath = '/mnt/d/NLeSC/DIANNA/data/mnist'
datapath = '../data/MNIST/raw'

In [ ]:
train_dataset = torchvision.datasets.MNIST(datapath, train=True, download=True)
test_dataset  = torchvision.datasets.MNIST(datapath, train=False, download=True)